# 模組 4: 端到端系統整合演示
## Module 4: End-to-End System Integration Demo

> **教學目標**: 整合前三個模組，建立完整的企業級資料治理系統，展示生產級部署能力

---

## 📚 學習成果 (Learning Outcomes)

完成本模組後，您將能夠：
1. **系統整合設計** - 將文檔處理、元資料管理、品質控制整合為統一系統
2. **生產級部署** - 實現 Docker 容器化、API 服務化、監控告警
3. **性能優化** - 大規模文檔處理的並發優化和資源管理
4. **運維管理** - 建立健康檢查、日誌分析、異常恢復機制

---

## 🎯 整合系統架構

### 完整資料流程

```mermaid
graph TB
    A[原始文檔] --> B[文檔攝取服務]
    B --> C[文檔處理引擎]
    C --> D[品質評估引擎]
    D --> E{品質檢查}
    E -->|通過| F[元資料管理]
    E -->|失敗| G[異常處理]
    F --> H[向量索引]
    H --> I[搜索引擎]
    I --> J[RAG 系統]
    
    G --> K[人工審核]
    K --> C
    
    L[監控系統] --> M[品質儀表板]
    L --> N[異常告警]
    
    style A fill:#e1f5ff
    style J fill:#e8f5e9
    style M fill:#fff3e0
    style N fill:#ffebee
```

### 核心組件

1. **文檔攝取層** - 多格式文檔的自動化攝取和預處理
2. **處理引擎層** - 語義分塊、內容提取、特徵生成  
3. **品質管控層** - 多維度品質評估、異常檢測、自動修復
4. **存儲管理層** - 元資料存儲、向量索引、版本控制
5. **服務介面層** - RESTful API、GraphQL、WebSocket
6. **監控運維層** - 健康檢查、性能監控、日誌分析

## 🔧 環境設定與依賴

In [ ]:
# 安裝完整系統依賴
!pip install fastapi uvicorn websockets
!pip install redis celery flower
!pip install prometheus-client grafana-api
!pip install docker python-multipart
!pip install aiofiles asyncpg
!pip install structlog rich

# 系統監控
!pip install psutil py-cpuinfo
!pip install healthcheck

In [ ]:
# 導入完整系統依賴
import os
import sys
import json
import asyncio
import logging
import multiprocessing
from pathlib import Path
from datetime import datetime, timedelta
from typing import List, Dict, Optional, Tuple, Any, AsyncGenerator
from dataclasses import dataclass, asdict
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import warnings
warnings.filterwarnings('ignore')

# Web 框架與 API
from fastapi import FastAPI, HTTPException, BackgroundTasks, Depends, File, UploadFile
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse, StreamingResponse
import uvicorn

# 非同步處理
import aiofiles
import aioredis
from celery import Celery

# 資料處理與分析
import pandas as pd
import numpy as np
from sqlalchemy.ext.asyncio import create_async_engine, AsyncSession
from sqlalchemy.orm import sessionmaker

# 監控與日誌
import structlog
from rich.console import Console
from rich.table import Table
from rich.progress import Progress, SpinnerColumn, TextColumn
from prometheus_client import Counter, Histogram, Gauge, generate_latest
import psutil
import py_cpuinfo

# 匯入前面模組的類別
from pathlib import Path
PROJECT_ROOT = Path('/home/os-sunnie.gd.weng/python_workstation/side-project/RAG/data_governance/kms_governance')
sys.path.append(str(PROJECT_ROOT))

# 設定結構化日誌
structlog.configure(
    processors=[
        structlog.stdlib.filter_by_level,
        structlog.stdlib.add_logger_name,
        structlog.stdlib.add_log_level,
        structlog.processors.TimeStamper(fmt="iso"),
        structlog.dev.ConsoleRenderer()
    ],
    wrapper_class=structlog.stdlib.BoundLogger,
    logger_factory=structlog.stdlib.LoggerFactory(),
    cache_logger_on_first_use=True,
)

logger = structlog.get_logger()
console = Console()

print("✅ 完整系統環境設定完成")
print(f"📂 專案根目錄: {PROJECT_ROOT}")
print(f"💻 CPU 核心數: {multiprocessing.cpu_count()}")
print(f"💾 系統記憶體: {psutil.virtual_memory().total / (1024**3):.1f} GB")

## 🏗️ 統一系統管理器

### 企業級文檔治理系統

In [ ]:
@dataclass
class SystemConfig:
    """系統配置"""
    # 資料庫配置
    database_url: str = "sqlite+aiosqlite:///./kms_governance.db"
    vector_db_path: str = "./vector_index"
    
    # 處理配置
    max_workers: int = 4
    chunk_size: int = 512
    chunk_overlap: int = 50
    
    # 品質配置
    min_quality_threshold: float = 0.7
    enable_auto_correction: bool = True
    
    # API 配置
    api_host: str = "0.0.0.0"
    api_port: int = 8000
    enable_cors: bool = True
    
    # 監控配置
    enable_metrics: bool = True
    log_level: str = "INFO"
    health_check_interval: int = 30

class EnterpriseDocumentGovernanceSystem:
    """
    企業級文檔治理系統
    整合文檔處理、元資料管理、品質控制的完整解決方案
    """
    
    def __init__(self, config: SystemConfig):
        """初始化系統"""
        self.config = config
        self.is_initialized = False
        self.is_running = False
        
        # 系統組件
        self.document_processor = None
        self.metadata_manager = None
        self.vector_manager = None
        self.quality_assessor = None
        self.anomaly_detector = None
        
        # 執行器
        self.thread_executor = ThreadPoolExecutor(max_workers=config.max_workers)
        self.process_executor = ProcessPoolExecutor(max_workers=min(4, config.max_workers))
        
        # 指標收集
        self._init_metrics()
        
        # 系統狀態
        self.system_stats = {
            'start_time': datetime.now(),
            'documents_processed': 0,
            'quality_assessments': 0,
            'anomalies_detected': 0,
            'api_requests': 0
        }
        
        logger.info("企業級文檔治理系統初始化完成")
    
    def _init_metrics(self):
        """初始化 Prometheus 指標"""
        if not self.config.enable_metrics:
            return
            
        self.metrics = {
            'documents_processed': Counter(
                'documents_processed_total', 
                'Total number of documents processed',
                ['status', 'type']
            ),
            'processing_duration': Histogram(
                'document_processing_duration_seconds',
                'Time spent processing documents',
                ['stage']
            ),
            'quality_score': Histogram(
                'document_quality_score',
                'Document quality scores',
                buckets=[0.1, 0.3, 0.5, 0.7, 0.8, 0.9, 0.95, 1.0]
            ),
            'system_health': Gauge(
                'system_health_score',
                'Overall system health score'
            ),
            'active_connections': Gauge(
                'active_api_connections',
                'Number of active API connections'
            )
        }
    
    async def initialize_system(self) -> bool:
        """
        初始化所有系統組件
        """
        try:
            logger.info("正在初始化系統組件...")
            
            with Progress(
                SpinnerColumn(),
                TextColumn("[progress.description]{task.description}"),
                console=console
            ) as progress:
                
                # 1. 初始化文檔處理器
                task1 = progress.add_task("初始化文檔處理器...", total=None)
                # 這裡需要實際的實作，簡化為模擬
                await asyncio.sleep(1)
                self.document_processor = "DocumentProcessor(config)"
                progress.update(task1, description="✅ 文檔處理器已就緒")
                
                # 2. 初始化元資料管理器
                task2 = progress.add_task("初始化元資料管理器...", total=None)
                await asyncio.sleep(1)
                self.metadata_manager = "MetadataManager(config.database_url)"
                progress.update(task2, description="✅ 元資料管理器已就緒")
                
                # 3. 初始化向量索引
                task3 = progress.add_task("初始化向量索引...", total=None)
                await asyncio.sleep(1)
                self.vector_manager = "VectorManager(config.vector_db_path)"
                progress.update(task3, description="✅ 向量索引已就緒")
                
                # 4. 初始化品質評估器
                task4 = progress.add_task("初始化品質評估器...", total=None)
                await asyncio.sleep(1)
                self.quality_assessor = "QualityAssessor()"
                progress.update(task4, description="✅ 品質評估器已就緒")
                
                # 5. 初始化異常檢測器
                task5 = progress.add_task("初始化異常檢測器...", total=None)
                await asyncio.sleep(1)
                self.anomaly_detector = "AnomalyDetector(config)"
                progress.update(task5, description="✅ 異常檢測器已就緒")
            
            self.is_initialized = True
            logger.info("所有系統組件初始化完成")
            return True
            
        except Exception as e:
            logger.error("系統初始化失敗", error=str(e))
            return False
    
    async def process_document_batch(self, 
                                   file_paths: List[str],
                                   metadata_list: List[Dict] = None) -> Dict:
        """
        批次處理文檔
        
        Args:
            file_paths: 文檔路徑列表
            metadata_list: 對應的元資料列表
            
        Returns:
            Dict: 批次處理結果
        """
        if not self.is_initialized:
            raise RuntimeError("系統尚未初始化")
        
        logger.info(f"開始批次處理 {len(file_paths)} 個文檔")
        
        results = {
            'processed': [],
            'failed': [],
            'quality_issues': [],
            'anomalies': [],
            'summary': {
                'total': len(file_paths),
                'success_count': 0,
                'failure_count': 0,
                'avg_quality_score': 0.0,
                'processing_time': 0.0
            }
        }
        
        start_time = datetime.now()
        
        # 並發處理文檔
        tasks = []
        for i, file_path in enumerate(file_paths):
            metadata = metadata_list[i] if metadata_list else {}
            task = self._process_single_document(file_path, metadata)
            tasks.append(task)
        
        # 等待所有任務完成
        with Progress(console=console) as progress:
            task_id = progress.add_task("處理文檔中...", total=len(tasks))
            
            for i, task in enumerate(asyncio.as_completed(tasks)):
                try:
                    result = await task
                    
                    if result['status'] == 'success':
                        results['processed'].append(result)
                        results['summary']['success_count'] += 1
                        
                        # 品質檢查
                        if result.get('quality_score', 1.0) < self.config.min_quality_threshold:
                            results['quality_issues'].append(result)
                            
                    else:
                        results['failed'].append(result)
                        results['summary']['failure_count'] += 1
                        
                except Exception as e:
                    logger.error(f"文檔處理失敗: {e}")
                    results['failed'].append({
                        'file_path': file_paths[i] if i < len(file_paths) else 'unknown',
                        'error': str(e),
                        'status': 'error'
                    })
                    results['summary']['failure_count'] += 1
                
                progress.update(task_id, advance=1)
        
        # 計算統計資訊
        end_time = datetime.now()
        processing_time = (end_time - start_time).total_seconds()
        
        if results['processed']:
            avg_quality = np.mean([r.get('quality_score', 0) for r in results['processed']])
            results['summary']['avg_quality_score'] = avg_quality
        
        results['summary']['processing_time'] = processing_time
        
        # 更新系統統計
        self.system_stats['documents_processed'] += results['summary']['success_count']
        
        # 更新指標
        if self.config.enable_metrics:
            self.metrics['documents_processed'].labels(
                status='success', type='batch'
            ).inc(results['summary']['success_count'])
            
            self.metrics['processing_duration'].labels(
                stage='batch_processing'
            ).observe(processing_time)
        
        logger.info(f"批次處理完成: {results['summary']}")
        return results
    
    async def _process_single_document(self, file_path: str, metadata: Dict) -> Dict:
        """
        處理單一文檔
        """
        try:
            start_time = datetime.now()
            
            # 模擬文檔處理過程
            await asyncio.sleep(0.1)  # 模擬處理時間
            
            # 模擬處理結果
            result = {
                'file_path': file_path,
                'document_id': f"doc_{hash(file_path) % 10000}",
                'status': 'success',
                'quality_score': np.random.uniform(0.3, 0.95),
                'chunk_count': np.random.randint(3, 15),
                'processing_time': (datetime.now() - start_time).total_seconds(),
                'metadata': metadata
            }
            
            return result
            
        except Exception as e:
            return {
                'file_path': file_path,
                'status': 'failed',
                'error': str(e),
                'processing_time': (datetime.now() - start_time).total_seconds()
            }
    
    async def get_system_health(self) -> Dict:
        """
        獲取系統健康狀態
        """
        # 獲取系統資源使用情況
        cpu_usage = psutil.cpu_percent(interval=1)
        memory = psutil.virtual_memory()
        disk = psutil.disk_usage('/')
        
        # 計算系統健康分數
        health_score = self._calculate_health_score(cpu_usage, memory.percent, disk.percent)
        
        uptime = datetime.now() - self.system_stats['start_time']
        
        health_status = {
            'status': 'healthy' if health_score > 0.8 else 'warning' if health_score > 0.6 else 'critical',
            'health_score': health_score,
            'uptime_seconds': uptime.total_seconds(),
            'system_resources': {
                'cpu_usage_percent': cpu_usage,
                'memory_usage_percent': memory.percent,
                'disk_usage_percent': disk.percent,
                'available_memory_gb': memory.available / (1024**3)
            },
            'component_status': {
                'document_processor': 'ready' if self.document_processor else 'not_initialized',
                'metadata_manager': 'ready' if self.metadata_manager else 'not_initialized',
                'vector_manager': 'ready' if self.vector_manager else 'not_initialized',
                'quality_assessor': 'ready' if self.quality_assessor else 'not_initialized',
                'anomaly_detector': 'ready' if self.anomaly_detector else 'not_initialized'
            },
            'statistics': self.system_stats.copy(),
            'timestamp': datetime.now().isoformat()
        }
        
        # 更新健康指標
        if self.config.enable_metrics:
            self.metrics['system_health'].set(health_score)
        
        return health_status
    
    def _calculate_health_score(self, cpu: float, memory: float, disk: float) -> float:
        """
        計算系統健康分數
        """
        # 簡單的健康評分算法
        cpu_score = max(0, (100 - cpu) / 100)
        memory_score = max(0, (100 - memory) / 100) 
        disk_score = max(0, (100 - disk) / 100)
        
        # 加權平均
        health_score = 0.4 * cpu_score + 0.4 * memory_score + 0.2 * disk_score
        
        # 如果系統未初始化，降低健康分數
        if not self.is_initialized:
            health_score *= 0.5
        
        return health_score
    
    async def shutdown(self):
        """優雅關閉系統"""
        logger.info("正在關閉系統...")
        
        self.is_running = False
        
        # 關閉執行器
        self.thread_executor.shutdown(wait=True)
        self.process_executor.shutdown(wait=True)
        
        logger.info("系統已安全關閉")

print("✅ 企業級文檔治理系統定義完成")

## 🌐 RESTful API 服務

### FastAPI 應用建立

In [ ]:
# 建立 FastAPI 應用
def create_app(system: EnterpriseDocumentGovernanceSystem) -> FastAPI:
    """
    建立 FastAPI 應用
    """
    app = FastAPI(
        title="企業 KMS 資料治理系統",
        description="Enterprise Knowledge Management System Data Governance API",
        version="1.0.0",
        docs_url="/docs",
        redoc_url="/redoc"
    )
    
    # CORS 設定
    if system.config.enable_cors:
        app.add_middleware(
            CORSMiddleware,
            allow_origins=["*"],
            allow_credentials=True,
            allow_methods=["*"],
            allow_headers=["*"],
        )
    
    # 健康檢查端點
    @app.get("/health")
    async def health_check():
        """系統健康檢查"""
        health_status = await system.get_system_health()
        
        status_code = 200
        if health_status['status'] == 'critical':
            status_code = 503
        elif health_status['status'] == 'warning':
            status_code = 200
            
        return JSONResponse(
            status_code=status_code,
            content=health_status
        )
    
    # 系統初始化端點
    @app.post("/system/initialize")
    async def initialize_system():
        """初始化系統"""
        if system.is_initialized:
            return {"status": "already_initialized", "message": "系統已經初始化"}
        
        success = await system.initialize_system()
        
        if success:
            return {"status": "success", "message": "系統初始化成功"}
        else:
            raise HTTPException(status_code=500, detail="系統初始化失敗")
    
    # 文檔上傳與處理端點
    @app.post("/documents/upload")
    async def upload_documents(files: List[UploadFile] = File(...)):
        """上傳並處理文檔"""
        if not system.is_initialized:
            raise HTTPException(status_code=503, detail="系統尚未初始化")
        
        # 儲存上傳的檔案
        upload_dir = Path("./uploads")
        upload_dir.mkdir(exist_ok=True)
        
        saved_files = []
        for file in files:
            file_path = upload_dir / file.filename
            
            async with aiofiles.open(file_path, 'wb') as f:
                content = await file.read()
                await f.write(content)
            
            saved_files.append(str(file_path))
        
        # 處理文檔
        results = await system.process_document_batch(saved_files)
        
        return {
            "status": "success",
            "message": f"已處理 {len(files)} 個文檔",
            "results": results
        }
    
    # 批次處理端點
    @app.post("/documents/batch-process")
    async def batch_process_documents(request: Dict):
        """批次處理文檔"""
        if not system.is_initialized:
            raise HTTPException(status_code=503, detail="系統尚未初始化")
        
        file_paths = request.get('file_paths', [])
        metadata_list = request.get('metadata_list', [])
        
        if not file_paths:
            raise HTTPException(status_code=400, detail="未提供檔案路徑")
        
        results = await system.process_document_batch(file_paths, metadata_list)
        
        return {
            "status": "success",
            "results": results
        }
    
    # 搜索端點
    @app.get("/search")
    async def search_documents(q: str, limit: int = 10, min_score: float = 0.0):
        """搜索文檔"""
        if not system.is_initialized:
            raise HTTPException(status_code=503, detail="系統尚未初始化")
        
        # 模擬搜索結果
        mock_results = [
            {
                "document_id": f"doc_{i}",
                "title": f"Document {i} related to {q}",
                "score": np.random.uniform(min_score, 1.0),
                "snippet": f"This document contains information about {q}..."
            }
            for i in range(min(limit, 5))
        ]
        
        return {
            "query": q,
            "results": mock_results,
            "total_count": len(mock_results)
        }
    
    # 指標端點
    @app.get("/metrics")
    async def get_metrics():
        """Prometheus 指標"""
        if not system.config.enable_metrics:
            raise HTTPException(status_code=404, detail="指標收集未啟用")
        
        return StreamingResponse(
            iter([generate_latest().decode('utf-8')]),
            media_type="text/plain"
        )
    
    # 系統統計端點
    @app.get("/system/stats")
    async def get_system_stats():
        """獲取系統統計資訊"""
        health_status = await system.get_system_health()
        
        return {
            "system_health": health_status,
            "performance_stats": {
                "documents_processed_per_minute": system.system_stats['documents_processed'] / max(1, health_status['uptime_seconds'] / 60),
                "average_processing_time": "N/A",  # 需要實際計算
                "success_rate": "N/A"  # 需要實際計算
            }
        }
    
    return app

print("✅ FastAPI 應用建立完成")

## 🧪 完整系統演示

### 系統啟動與測試

In [ ]:
async def run_system_demo():
    """
    運行完整系統演示
    """
    console.print("\n🚀 [bold blue]企業級 KMS 資料治理系統演示[/bold blue]\n")
    
    # 1. 系統配置
    config = SystemConfig(
        max_workers=4,
        min_quality_threshold=0.6,
        enable_metrics=True
    )
    
    # 2. 初始化系統
    console.print("📋 [yellow]步驟 1: 初始化企業治理系統[/yellow]")
    governance_system = EnterpriseDocumentGovernanceSystem(config)
    
    init_success = await governance_system.initialize_system()
    if not init_success:
        console.print("❌ [red]系統初始化失敗[/red]")
        return
    
    # 3. 建立測試文檔
    console.print("\n📋 [yellow]步驟 2: 建立測試文檔[/yellow]")
    
    test_dir = Path("./demo_documents")
    test_dir.mkdir(exist_ok=True)
    
    demo_documents = {
        "enterprise_ai_strategy.txt": """
Enterprise AI Strategy 2024-2026

Executive Summary
This document outlines our comprehensive artificial intelligence strategy for the next three years. We aim to leverage AI technologies to enhance operational efficiency, improve customer experience, and drive innovation across all business units.

Strategic Objectives
1. Implement AI-powered automation in 70% of routine processes by 2025
2. Deploy machine learning models for predictive analytics
3. Establish enterprise-wide data governance framework
4. Build AI talent pool through training and recruitment

Implementation Roadmap
Phase 1 (2024): Foundation building and pilot projects
Phase 2 (2025): Scaling successful initiatives
Phase 3 (2026): Full enterprise integration and optimization

Risk Mitigation
We acknowledge potential risks including data privacy concerns, algorithmic bias, and workforce displacement. Our mitigation strategies include comprehensive governance policies, regular audits, and employee retraining programs.
""",
        "technical_architecture.txt": """
Microservices Architecture Design Document

Overview
This document describes the technical architecture for our new microservices-based platform. The architecture follows domain-driven design principles and implements event-driven communication patterns.

Core Services
- User Management Service: Handles authentication and authorization
- Document Processing Service: Manages file upload and content extraction
- Search Service: Provides full-text and semantic search capabilities
- Notification Service: Handles all system notifications and alerts

Technology Stack
- Container Platform: Docker + Kubernetes
- API Gateway: Kong
- Message Queue: Apache Kafka
- Databases: PostgreSQL, Redis, Elasticsearch
- Monitoring: Prometheus + Grafana

Security Considerations
All services implement OAuth 2.0 authentication, API rate limiting, and comprehensive audit logging. Data encryption is enforced both at rest and in transit.
""",
        "quality_metrics_report.txt": """
Q3 2024 Quality Metrics Report

Document Processing Quality
During Q3 2024, our document governance system processed 15,847 documents with an average quality score of 0.847. This represents a 12% improvement over Q2 2024.

Key Performance Indicators
- Processing Success Rate: 98.2%
- Average Processing Time: 2.3 seconds per document
- Quality Threshold Compliance: 92%
- Anomaly Detection Accuracy: 94.5%

Quality Dimensions Analysis
Accuracy: 0.89 (excellent)
Completeness: 0.85 (good)
Consistency: 0.82 (good)
Currency: 0.78 (fair)
Understandability: 0.91 (excellent)
Traceability: 0.84 (good)

Recommendations
1. Focus on improving document currency through automated update detection
2. Enhance consistency checking algorithms for technical documentation
3. Implement proactive quality monitoring for real-time feedback
"""
    }
    
    file_paths = []
    for filename, content in demo_documents.items():
        file_path = test_dir / filename
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(content)
        file_paths.append(str(file_path))
    
    console.print(f"✅ 建立了 {len(demo_documents)} 個測試文檔")
    
    # 4. 批次處理演示
    console.print("\n📋 [yellow]步驟 3: 批次處理文檔[/yellow]")
    
    # 準備元資料
    metadata_list = [
        {
            "title": "Enterprise AI Strategy 2024-2026",
            "authors": ["Strategy Team"],
            "document_type": "strategic_plan",
            "department": "Corporate Strategy"
        },
        {
            "title": "Microservices Architecture Design Document",
            "authors": ["Architecture Team"],
            "document_type": "technical_spec",
            "department": "Engineering"
        },
        {
            "title": "Q3 2024 Quality Metrics Report",
            "authors": ["Quality Assurance Team"],
            "document_type": "quality_report",
            "department": "Operations"
        }
    ]
    
    # 執行批次處理
    batch_results = await governance_system.process_document_batch(file_paths, metadata_list)
    
    # 5. 展示處理結果
    console.print("\n📋 [yellow]步驟 4: 處理結果分析[/yellow]")
    
    # 建立結果表格
    results_table = Table(
        title="批次處理結果",
        show_header=True,
        header_style="bold magenta"
    )
    
    results_table.add_column("文檔", style="cyan")
    results_table.add_column("狀態", style="green")
    results_table.add_column("品質分數", style="yellow")
    results_table.add_column("分塊數", style="blue")
    results_table.add_column("處理時間", style="magenta")
    
    for result in batch_results['processed']:
        filename = Path(result['file_path']).name
        status_icon = "✅" if result['status'] == 'success' else "❌"
        quality_score = f"{result['quality_score']:.3f}"
        chunk_count = str(result['chunk_count'])
        processing_time = f"{result['processing_time']:.2f}s"
        
        results_table.add_row(
            filename, 
            f"{status_icon} {result['status']}", 
            quality_score,
            chunk_count,
            processing_time
        )
    
    console.print(results_table)
    
    # 6. 系統健康狀態
    console.print("\n📋 [yellow]步驟 5: 系統健康檢查[/yellow]")
    
    health_status = await governance_system.get_system_health()
    
    health_table = Table(
        title="系統健康狀態",
        show_header=True,
        header_style="bold blue"
    )
    
    health_table.add_column("項目", style="cyan")
    health_table.add_column("狀態/數值", style="green")
    
    health_table.add_row("整體狀態", health_status['status'].upper())
    health_table.add_row("健康分數", f"{health_status['health_score']:.3f}")
    health_table.add_row("運行時間", f"{health_status['uptime_seconds']:.0f} 秒")
    health_table.add_row("CPU 使用率", f"{health_status['system_resources']['cpu_usage_percent']:.1f}%")
    health_table.add_row("記憶體使用率", f"{health_status['system_resources']['memory_usage_percent']:.1f}%")
    health_table.add_row("已處理文檔", str(health_status['statistics']['documents_processed']))
    
    console.print(health_table)
    
    # 7. 摘要統計
    console.print("\n📊 [bold green]演示摘要[/bold green]")
    summary = batch_results['summary']
    
    console.print(f"• 總處理文檔數: {summary['total']}")
    console.print(f"• 成功處理: {summary['success_count']}")
    console.print(f"• 處理失敗: {summary['failure_count']}")
    console.print(f"• 平均品質分數: {summary['avg_quality_score']:.3f}")
    console.print(f"• 總處理時間: {summary['processing_time']:.2f} 秒")
    console.print(f"• 平均處理速度: {summary['total']/summary['processing_time']:.1f} 文檔/秒")
    
    if batch_results['quality_issues']:
        console.print(f"• ⚠️  品質問題文檔: {len(batch_results['quality_issues'])} 個")
    
    console.print("\n🎉 [bold green]企業級 KMS 資料治理系統演示完成！[/bold green]")
    
    # 8. 關閉系統
    await governance_system.shutdown()

# 運行演示
print("🚀 準備運行完整系統演示...")
await run_system_demo()

### API 服務啟動演示

In [ ]:
async def demo_api_service():
    """
    演示 API 服務功能
    """
    console.print("\n🌐 [bold blue]API 服務演示[/bold blue]\n")
    
    # 初始化系統
    config = SystemConfig(api_port=8001)  # 使用不同端口避免衝突
    governance_system = EnterpriseDocumentGovernanceSystem(config)
    await governance_system.initialize_system()
    
    # 建立 FastAPI 應用
    app = create_app(governance_system)
    
    console.print("📋 API 端點清單:")
    
    api_endpoints = [
        ("GET", "/health", "系統健康檢查"),
        ("POST", "/system/initialize", "初始化系統"),
        ("POST", "/documents/upload", "上傳文檔"),
        ("POST", "/documents/batch-process", "批次處理"),
        ("GET", "/search", "搜索文檔"),
        ("GET", "/metrics", "Prometheus 指標"),
        ("GET", "/system/stats", "系統統計")
    ]
    
    endpoints_table = Table(
        title="API 端點",
        show_header=True,
        header_style="bold cyan"
    )
    
    endpoints_table.add_column("方法", style="green")
    endpoints_table.add_column("路徑", style="blue")
    endpoints_table.add_column("說明", style="yellow")
    
    for method, path, description in api_endpoints:
        endpoints_table.add_row(method, path, description)
    
    console.print(endpoints_table)
    
    # 模擬 API 測試
    console.print("\n📋 [yellow]模擬 API 測試[/yellow]")
    
    # 模擬健康檢查
    health_response = await governance_system.get_system_health()
    console.print(f"✅ GET /health - 狀態: {health_response['status']}")
    
    # 模擬批次處理請求
    mock_request = {
        "file_paths": ["./demo_documents/enterprise_ai_strategy.txt"],
        "metadata_list": [{
            "title": "Enterprise AI Strategy",
            "department": "Strategy"
        }]
    }
    
    batch_response = await governance_system.process_document_batch(
        mock_request['file_paths'],
        mock_request['metadata_list']
    )
    
    console.print(f"✅ POST /documents/batch-process - 處理了 {batch_response['summary']['success_count']} 個文檔")
    
    # 顯示 API 配置
    console.print("\n📋 API 服務配置:")
    console.print(f"• 主機: {config.api_host}")
    console.print(f"• 端口: {config.api_port}")
    console.print(f"• CORS: {'啟用' if config.enable_cors else '禁用'}")
    console.print(f"• 指標收集: {'啟用' if config.enable_metrics else '禁用'}")
    console.print(f"• 文檔: http://{config.api_host}:{config.api_port}/docs")
    
    console.print("\n💡 [bold yellow]啟動 API 服務命令:[/bold yellow]")
    console.print(f"uvicorn main:app --host {config.api_host} --port {config.api_port} --reload")
    
    await governance_system.shutdown()

# 運行 API 演示
await demo_api_service()

## 📊 性能分析與監控

### 系統性能評估

In [ ]:
async def performance_benchmark():
    """
    系統性能基準測試
    """
    console.print("\n⚡ [bold blue]系統性能基準測試[/bold blue]\n")
    
    # 建立測試配置
    test_configs = [
        SystemConfig(max_workers=1, chunk_size=256),
        SystemConfig(max_workers=2, chunk_size=512),
        SystemConfig(max_workers=4, chunk_size=512),
        SystemConfig(max_workers=8, chunk_size=1024)
    ]
    
    # 建立測試文檔
    test_docs = [f"test_doc_{i}.txt" for i in range(20)]
    test_dir = Path("./benchmark_docs")
    test_dir.mkdir(exist_ok=True)
    
    # 生成測試內容
    for doc_name in test_docs:
        content = f"""
Performance Test Document {doc_name}

This is a test document for performance benchmarking. It contains multiple paragraphs to simulate real-world document processing scenarios.

Introduction
Performance testing is crucial for ensuring system reliability and scalability. This document serves as a representative sample for measuring processing throughput and latency.

Content Section 1
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris.

Content Section 2
Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.

Conclusion
This document demonstrates the system's ability to process structured content with multiple sections and varying complexity levels.
"""
        
        file_path = test_dir / doc_name
        with open(file_path, 'w') as f:
            f.write(content)
    
    test_file_paths = [str(test_dir / doc) for doc in test_docs]
    
    # 性能測試結果
    performance_results = []
    
    for i, config in enumerate(test_configs):
        console.print(f"📊 測試配置 {i+1}: {config.max_workers} workers, {config.chunk_size} chunk size")
        
        # 初始化系統
        governance_system = EnterpriseDocumentGovernanceSystem(config)
        await governance_system.initialize_system()
        
        # 執行性能測試
        start_time = datetime.now()
        
        results = await governance_system.process_document_batch(test_file_paths)
        
        end_time = datetime.now()
        total_time = (end_time - start_time).total_seconds()
        
        # 收集性能指標
        performance_data = {
            'config': f"{config.max_workers}w_{config.chunk_size}c",
            'max_workers': config.max_workers,
            'chunk_size': config.chunk_size,
            'total_documents': len(test_file_paths),
            'total_time': total_time,
            'throughput': len(test_file_paths) / total_time,
            'avg_processing_time': total_time / len(test_file_paths),
            'success_count': results['summary']['success_count'],
            'avg_quality_score': results['summary']['avg_quality_score']
        }
        
        performance_results.append(performance_data)
        
        console.print(f"  ⏱️  總時間: {total_time:.2f}s")
        console.print(f"  📈 吞吐量: {performance_data['throughput']:.1f} 文檔/秒")
        console.print(f"  ✅ 成功率: {performance_data['success_count']}/{performance_data['total_documents']}")
        console.print()
        
        await governance_system.shutdown()
    
    # 性能結果表格
    perf_table = Table(
        title="性能基準測試結果",
        show_header=True,
        header_style="bold green"
    )
    
    perf_table.add_column("配置", style="cyan")
    perf_table.add_column("Worker數", style="blue")
    perf_table.add_column("分塊大小", style="yellow")
    perf_table.add_column("總時間(s)", style="magenta")
    perf_table.add_column("吞吐量(doc/s)", style="green")
    perf_table.add_column("平均品質", style="red")
    
    for result in performance_results:
        perf_table.add_row(
            result['config'],
            str(result['max_workers']),
            str(result['chunk_size']),
            f"{result['total_time']:.2f}",
            f"{result['throughput']:.1f}",
            f"{result['avg_quality_score']:.3f}"
        )
    
    console.print(perf_table)
    
    # 找出最佳配置
    best_config = max(performance_results, key=lambda x: x['throughput'])
    console.print(f"\n🏆 [bold green]最佳性能配置:[/bold green] {best_config['config']}")
    console.print(f"   吞吐量: {best_config['throughput']:.1f} 文檔/秒")
    console.print(f"   平均處理時間: {best_config['avg_processing_time']:.3f} 秒/文檔")
    
    # 性能建議
    console.print("\n💡 [bold yellow]性能優化建議:[/bold yellow]")
    
    if best_config['max_workers'] >= 4:
        console.print("• 系統適合高並發處理，建議在生產環境中使用多 worker 配置")
    else:
        console.print("• 系統在低並發下表現更佳，可能存在 I/O 瓶頸")
    
    avg_throughput = np.mean([r['throughput'] for r in performance_results])
    if avg_throughput > 5:
        console.print("• 系統整體性能良好，適合生產環境部署")
    else:
        console.print("• 建議進一步優化算法或升級硬件配置")
    
    console.print("\n🎉 性能基準測試完成！")

# 運行性能測試
await performance_benchmark()

## 🎓 課程總結與後續學習

### 完整學習成果檢核

🎉 恭喜！您已完成企業級 KMS 資料治理系統的完整學習旅程。

#### ✅ 四大核心模組掌握

**模組 1: 文檔攝取與處理**
- 企業級文檔處理架構設計
- Docling 深度整合與多格式支援
- 語義分塊演算法實作
- 自動化品質檢測機制

**模組 2: 元資料管理與索引**
- 可擴展的元資料 schema 設計
- 企業級向量索引系統
- 混合檢索策略實作
- 資料血緣追蹤機制

**模組 3: 品質控制與監控**
- 多維度品質評估框架 (ISO 25012)
- 智能異常檢測系統
- 自動化品質監控
- 品質治理最佳實踐

**模組 4: 端到端系統整合**
- 完整系統架構整合
- RESTful API 服務化
- 性能優化與監控
- 生產級部署準備

### 🚀 實際應用能力

完成本課程後，您具備了：

1. **系統設計能力** - 運用運算思維設計可擴展的治理架構
2. **技術實作能力** - 掌握從文檔處理到向量檢索的完整技術棧
3. **品質管理能力** - 建立企業級的資料品質保證體系
4. **運維監控能力** - 實現系統健康監控與性能優化

### 💼 職業應用場景

- **企業數據治理專家** - 建立組織級資料品質管理制度
- **RAG 系統架構師** - 設計高品質知識庫支撐的 AI 應用
- **文檔管理系統開發者** - 開發智能化文檔處理平台
- **AI 工程師** - 為 LLM 應用提供可信賴的知識基礎

### 🔮 下一步學習建議

#### 深度專業化方向

1. **RAG 系統實作** 
   - 學習 LangChain、LlamaIndex 框架
   - 實作檢索增強生成應用
   - 掌握 Prompt Engineering 技術

2. **大規模部署**
   - Kubernetes 容器編排
   - 微服務架構進階
   - 雲原生技術棧

3. **AI/ML 進階**
   - 自然語言處理深度學習
   - 向量數據庫優化
   - MLOps 工程實踐

#### 技術擴展領域

- **多模態處理**: 圖像、音頻、視頻內容治理
- **實時流處理**: Kafka、Flink 流式資料處理
- **安全與隱私**: 差分隱私、聯邦學習
- **可解釋 AI**: 模型解釋性與透明度

### 📚 推薦資源

#### 技術文檔
- [Docling 官方文檔](https://github.com/DS4SD/docling)
- [ChromaDB 向量數據庫](https://www.trychroma.com/)
- [FastAPI 框架](https://fastapi.tiangolo.com/)

#### 標準規範
- ISO/IEC 25012:2008 - 資料品質標準
- DAMA-DMBOK - 資料管理知識體系
- OpenAPI 3.0 規範

#### 開源專案
- [LangChain](https://github.com/langchain-ai/langchain) - LLM 應用框架
- [Weaviate](https://github.com/weaviate/weaviate) - 向量數據庫
- [Haystack](https://github.com/deepset-ai/haystack) - NLP 管線

---

## 🙏 結語

感謝您完成這個深入的學習旅程！您現在擁有了建立企業級資料治理系統的完整技能和知識。

記住，**品質是 AI 系統成功的基石**。在這個資訊爆炸的時代，能夠從海量資料中提取、管理和利用高品質知識的能力，將成為組織競爭力的關鍵。

希望您能將這些知識應用到實際專案中，為組織創造價值，為 AI 的健康發展貢獻力量！

🚀 **繼續學習，持續創新！**